# pyWECcast - v0.2.2
## WEC Powerseries Generator

This version release represents a refactoring to reduce dependence on specific file configurations for powerseries generation.

This is also the first implementation of numba accelerated functions with parallel functionality included

The example shows what may be a basic example of combining multiple sea-states


In [1]:
# General import directly from the local pyWECcast directory

import sys
sys.path.insert(1, '../')

import src as wc

In [2]:
baseDir = f'/mnt/c/Users/abharath/Documents/Projects/Grid_Value'
dataDir = f'{baseDir}/data'
buoyDB = f'{dataDir}/buoy_downloads.h5'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

In [3]:
# In this example we will combined both wind and swell waves to estimate the WEC power output 
# based on a fully realised sea

buoyNum = '46041'

time_index, SwH, SwP, SwD = wc.extract_NOAA_buoy(buoyDB,buoyNum,'SwH', 'SwP', 'SwD')
time_index, WWH, WWP, WWD = wc.extract_NOAA_buoy(buoyDB,buoyNum,'WWH', 'WWP', 'WWD')

print(SwH, SwP, SwD, WWH, WWP, WWD, time_index)

[3.4 3.  3.2 ... 2.  1.6 1.7] [11.4 10.8 11.4 ... 10.8 10.8 11.4] [b'WSW' b'SW' b'WSW' ... b'NW' b'WNW' b'NW'] [0.6 0.4 0.4 ... 0.3 0.3 0.3] [4.3 3.4 3.6 ... 3.6 3.7 4. ] [b'S' b'S' b'SSW' ... b'WNW' b'NW' b'NW'] [b'2020-01-29 00:00:00' b'2020-01-29 01:00:00' b'2020-01-29 02:00:00' ...
 b'2020-03-13 19:00:00' b'2020-03-13 20:00:00' b'2020-03-13 21:00:00']


In [4]:
# for the particular NOAA buoy data files we have convenience functions in pyWECcast.ufuncs 
# to convert to a numerical format.

timestamps = wc.ufuncs.binary_timestamp_convert(time_index)


In [5]:
# we can manipulate the data to reduuce computation. Note array shapes must remain equal

cut = 100
timestamps, SwH, SwP, WWH, WWP = timestamps[:cut], SwH[:cut], SwP[:cut], WWH[:cut], WWP[:cut]

In [6]:
# With a WECSim powerseries matrix, we can link the sea-state data to the WEC simulations 
# that are available. The closest match can be determined using a nearest neighbor method on 
# separate Hs, Tp and direction values or combinations of the groups

# Here we create links for both the wind and swell waves

wecSim = f'{dataDir}/WECSim_dataset_RM3_scale_0-1873.hdf5'

swell, measured = wc.link_sea_states(wecSim,SwH,SwP)
wind, measured = wc.link_sea_states(wecSim,WWH,WWP)


In [7]:
# With the sea-states in the dataset linked to what is available in the WECSim simulation db
# We can calculate the FFTs to obtian the reconstruction coefficients for both the wind and 
# swell waves

# if inMemory:True, calculate_fft_matrix() will return coefficients as a dict

swell_matrix = wc.calculate_fft_matrix(wecSim,swell[:,0],swell[:,1],inMemory=True)
wind_matrix = wc.calculate_fft_matrix(wecSim,wind[:,0],wind[:,1],inMemory=True)


In [8]:
# if we are looking to simulate multiple WECs, there is a convinience function in place to
# setup the fft_matrix to do this.

nWECs = 4

swell_matrix = wc.multiple_realizations(nWECs,swell_matrix)
wind_matrix = wc.multiple_realizations(nWECs,wind_matrix)


In [9]:
# With the coefficients and frequencies we can not reconstruct the powerseries for a number of 
# WECs and output frequencies

freq = '60S'

swell_result, time = wc.construct_powerseries(timestamps,freq,swell[:,0],swell[:,1],
                         fft_matrix=swell_matrix,inMemory=True)
wind_result, time = wc.construct_powerseries(timestamps,freq,wind[:,0],wind[:,1],
                         fft_matrix=wind_matrix,inMemory=True)

100%|██████████| 99/99 [00:13<00:00,  7.09it/s]


In [10]:
# to Naively combine power output from both wind and swell wave seas we simply sum the power
# generated from each individual sea

from numpy import sum, dstack

combination = dstack([swell_result,wind_result])
sumResults = sum(combination,axis=-1)

In [11]:
# Plots of the results can be generated to view the wind, swell and combine power outputs along
# with the means between any number of WECs

%matplotlib widget

import matplotlib.pyplot as plt
from numpy import mean

for i in range(swell_result.shape[-1]):
    plt.plot(time,swell_result[:,i])
    
for i in range(wind_result.shape[-1]):
    plt.plot(time,wind_result[:,i])

for i in range(sumResults.shape[-1]):
    plt.plot(time,sumResults[:,i])
    
plt.plot(time,mean(sumResults,axis=1),color='k')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …